In [86]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [124]:
# Constants
NUM_GAMES = 1
BATCH_SIZE = 32
LR = 1e-3

In [125]:
df = pd.read_csv('champions.csv')
def get_champ_vec(champ_name):
    arr = []
    row = df.loc[df['Champion'] == champ_name].iloc[0]
    for i in range(1, 16):
        if i == 7:
            if row[i] == 1:
                arr.append(1)
                arr.append(0)
                arr.append(0)
            elif row[i] == 2:
                arr.append(0)
                arr.append(1)
                arr.append(0)
            elif row[i] == 3:
                arr.append(0)
                arr.append(0)
                arr.append(1)
        else:
            arr.append(row[i] / 10)
    return arr

In [126]:
def process_match(match_index):
    with open('data/matches/' + str(match_index) + '.txt', 'r') as f:
        rows = f.read().split('\n')
        vec = []
        for j in range(10):
            champ_vec = get_champ_vec(rows[j])
            for e in champ_vec:
                vec.append(e)
        if rows[10] == 'true':
            vec.append(1)
        else:
            vec.append(0)
        return vec

In [127]:
def process_row(x):
    return (tf.cast(x[:-1], 'float32'), tf.cast(x[-1], 'float32'))

In [128]:
def get_dataset():
    tmp = []
    for i in range(NUM_GAMES):
        tmp.append(process_match(i))
    ds = tf.data.Dataset.from_tensor_slices(tmp)
    ds = ds.map(process_row)
    ds = ds.cache()
    ds = ds.shuffle(NUM_GAMES)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch()
    return ds

In [129]:
ds = get_dataset()

TypeError: DatasetV2.prefetch() missing 1 required positional argument: 'buffer_size'

In [123]:
for i in ds:
    print(i)

(<tf.Tensor: shape=(170,), dtype=float32, numpy=
array([0.7, 0.8, 0.3, 0.7, 0.9, 0.6, 0. , 1. , 0. , 0. , 0. , 0.5, 0.7,
       0.2, 0.6, 0.9, 0. , 0.7, 0.8, 0.8, 0.1, 0.7, 0.6, 1. , 0. , 0. ,
       0.1, 0.6, 0.7, 0.6, 0.8, 0.8, 0.6, 0.6, 0.4, 1. , 0.2, 0.1, 0.8,
       0.5, 0. , 1. , 0. , 0. , 0.1, 0.8, 0.4, 0.3, 0.5, 0. , 0. , 0.1,
       0.9, 0.4, 0.9, 0.6, 0.2, 1. , 0. , 0. , 0.4, 0. , 0. , 0.3, 1. ,
       0.3, 0. , 0.3, 0.8, 0.8, 0.7, 0.8, 0.6, 0.7, 1. , 0. , 0. , 0. ,
       0. , 1. , 0.4, 0.5, 0.8, 0. , 0.7, 0.3, 0.6, 0. , 0.3, 0.6, 0.6,
       0. , 0. , 1. , 0. , 0. , 0. , 0.4, 0.2, 0. , 0.8, 0. , 0.5, 0.4,
       0. , 0.6, 0.1, 0.4, 1. , 0. , 0. , 0. , 0. , 0.2, 0.1, 0. , 0. ,
       0.5, 0. , 0.5, 0.1, 0.5, 0.9, 0.4, 0.5, 1. , 0. , 0. , 0. , 0. ,
       0.8, 0.6, 0. , 0.6, 0.5, 0.3, 0.9, 0.5, 0.5, 0.8, 0.4, 0. , 0. ,
       1. , 0. , 0. , 0.6, 0.8, 0.3, 0. , 0.9, 0.4, 0. , 0.5, 0.8, 0.6,
       0.8, 0.2, 0.8, 1. , 0. , 0. , 0. , 0.2, 0.8, 0.4, 0.4, 0.7, 0.2,
       0.2], dt